# Take Out Unnecesary Columns for Exclusively Translation extraction

In [2]:
import pandas as pd
import numpy as np
import json
from dutchanalyzer.config import *
from dutchanalyzer.utils import *
from pathlib import Path
from dotenv import load_dotenv
from io import StringIO
import datetime
import re

In [3]:
previous_save_folder = Path(INTERIM_DATA_DIR, 'preprocessing', 'wikt', '31-10-25')
en_en_concat_folder = Path(previous_save_folder, 'concat_dfs_en_en')
en_nl_concat_folder = Path(previous_save_folder, 'concat_dfs_en_nl')
nl_en_concat_folder = Path(previous_save_folder, 'concat_dfs_nl_en')
nl_nl_concat_folder = Path(previous_save_folder, 'concat_dfs_nl_nl')
stripped_dfs_folder = Path(previous_save_folder, 'stripped_dfs')

raw_en_concat_folder = Path(previous_save_folder, 'concat_dfs_raw_en')
raw_nl_concat_folder = Path(previous_save_folder, 'concat_dfs_raw_nl')

In [13]:
def make_folder(parent_folder_path, folder_dict):
    path_list = []
    if not parent_folder_path.exists():
        Path.mkdir(parent_folder_path, parents=True, exist_ok=True)
        path_list.append(parent_folder_path)
    for k, v in folder_dict.items():
        Path.mkdir(Path(parent_folder_path, k), parents=True, exist_ok=True)
        path_list.append(Path(parent_folder_path, k))
        if type(v) == list:
            for i in v:
                if type(i) == str:
                    Path.mkdir(Path(parent_folder_path, k, i), parents=True, exist_ok=True)
                    path_list.append(Path(parent_folder_path, k, i))
                elif type(i) == dict:
                    pl = make_folder(Path(parent_folder_path, k), i)
                    path_list.append(pl)
        elif type(v) == dict:    
            pl = make_folder(Path(parent_folder_path, k), i)
            path_list.append(pl)
    return path_list


In [ ]:
today = datetime.date.today().__format__("%d-%m-%y")
current_save_folder = Path(INTERIM_DATA_DIR, 'preprocessing', 'wikt', str(today))
folders = {'en': ['pp_en', 'r_en', 'pp_nl', 'r_nl'], 'nl':['pp_en', 'r_en', 'pp_nl', 'r_nl']}
path_list = make_folder(current_save_folder, folders)

In [16]:
for p in path_list: print(p)

C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim\preprocessing\wikt\03-11-25\en
C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim\preprocessing\wikt\03-11-25\en\pp_en
C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim\preprocessing\wikt\03-11-25\en\r_en
C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim\preprocessing\wikt\03-11-25\en\pp_nl
C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim\preprocessing\wikt\03-11-25\en\r_nl
C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim\preprocessing\wikt\03-11-25\nl
C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim\preprocessing\wikt\03-11-25\nl\pp_en
C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim\preprocessing\wikt\03-11-25\nl\r_en
C:\Users\elise\SynologyDrive\Dev\DutchAnalyzerPublic\DutchAnalyzer\data\interim

In [17]:
en_folder = path_list[0]
en_nl_pp_folder = path_list[3]
nl_folder = path_list[5]
nl_nl_pp_folder = path_list[6]

In [18]:
pp_enwikt_en_path = Path(en_en_concat_folder, 'en_en_json_pages_0-711977.csv')
pp_enwikt_nl_path = Path(en_nl_concat_folder, 'en_nl_json_pages_0-70461.csv')
pp_nlwikt_en_path = Path(nl_en_concat_folder, 'nl_en_json_pages_0-8720.csv')
pp_nlwikt_nl_path = Path(nl_nl_concat_folder, 'nl_nl_json_pages_0-305722.csv')

In [19]:
r_enwikt_en_df_path = Path(raw_en_concat_folder, 'r_enwikt_en_df.csv')
r_enwikt_nl_df_path = Path(raw_en_concat_folder, 'r_enwikt_nl_df.csv')
r_nlwikt_en_df_path = Path(raw_nl_concat_folder, 'r_nlwikt_en_df.csv')
r_nlwikt_nl_df_path = Path(raw_nl_concat_folder, 'r_nlwikt_nl_df.csv')

### Notes on Usage
- EEP - Table 1 will be en-en pp
- ENP - en-nl pp
- EER - en-en raw
- ENR - en-nl raw
- NEP
- NNP
- NER
- NNR

In [ ]:
"""
The following columns will be dropped and rows in general:

Then IDs added
"""

In [138]:
def add_table_id(df, wikt_lang, table_lang, data_source_type='P', alt_id_name=''):
    words = df['word'].unique()
    try:
        words.sort()
    except:
        try:
            words.sort_values()
        except:
            print('problem')
    word_ids = [x for x in range(len(words))]
    id_name = ''
    if alt_id_name == '':
        id_name += wikt_lang[0].upper()
        id_name += table_lang[0].upper()
        id_name += data_source_type
        id_name += '_id'
    else:
        id_name = alt_id_name

    id_df = pd.DataFrame({id_name: word_ids, 'word':words})
    id_df = id_df.merge(df, how='outer', on='word')
    return id_df
    

In [71]:
def add_table_wikt_cols(df, wikt_lang, table_lang, data_source_type='P', alt_source_id_name=''):
    dscr = ''
    if alt_source_id_name == '':
        dscr += wikt_lang[0].upper()
        dscr += table_lang[0].upper()
        dscr += data_source_type
        dscr += '_src'
    else:
        dscr = alt_source_id_name
    dsc = [dscr for x in range(len(df['word']))]    
    df['dsrc'] = dsc
    return df

In [120]:
def sort_columns(df, start_cols=['word_id', 'word', 'pos', 'lang_code'], end_cols=['dsrc'], subsort_categories=False, sorted_categories={}, col_end_sort='word'):
    df_cols = df.columns
    df_cols = df_cols.sort_values()
    e_cols = [x for x in end_cols if x in df_cols]
    s_cols = [x for x in start_cols if x in df_cols]
    word_id_col = ''
    if 'word_id' in start_cols and 'word_id' not in df_cols:
        sub_ids = ['EEP_id', 'ENP_id', 'EER_id', 'ENR_id', 'NEP_id', 'NNP_id', 'NER_id', 'NNR_id']
        for i in sub_ids:
            if i in df_cols:
                s_cols = [i] + s_cols
                word_id_col = i
    elif 'word_id' in df_cols:
        word_id_col = 'word_id'
    reserved_cols = s_cols + e_cols
    df_cols = [x for x in df_cols if x not in reserved_cols]
    df_cols.sort()
    new_cols = s_cols + df_cols + e_cols

    if subsort_categories == False:
        new_cols = s_cols + df_cols + e_cols
        print(new_cols)
    else:
        print('not implemented')
        new_cols = s_cols + df_cols + e_cols
    

    df = df.loc[:, new_cols]
    
    if col_end_sort == 'word_id':
        df = df.sort_values(by=word_id_col)
    elif col_end_sort == 'word':
        df = df.sort_values(by='word')

    return df

In [144]:
def process_table(df, cols_to_drop=['abbreviations', 'anagrams', 'etymology_number', 'form_of', 'info_templates', 'alt_of', 'wikidata', 'pos_title', 'source', 'wikipedia', 'proverbs', 'lang'], col_vals_to_drop={'pos':['proverb', 'phrase', 'character', 'symbol', 'punct', 'abbrev']}, starting_cols=['word', 'pos', 'lang_code']):
    print("Shape before: ", df.shape)
    df = return_non_na(df, 'word')
    rel_drop_cols = [x for x in cols_to_drop if x in df.columns]
    has_invalid = []
    df = df.drop(columns=rel_drop_cols)
    for k in col_vals_to_drop.keys():
        if k in df.columns:
           vals = col_vals_to_drop[k]
           for v in vals:
               df = df[df[k] != v]
    
    for i, row in df.iterrows():
        w = row['word']
        is_invalid = check_has_invalid_chars(w)
        has_invalid.append(is_invalid)
    df['invalid'] = has_invalid
    df = df[df['invalid'] != True]
    df = df.drop(columns=['invalid'])
    if 'etymology_text' in df.columns:
        df.rename(columns={'etymology_text': 'etymology_texts'})
    print("Shape after: ", df.shape)
    return df

In [162]:
def full_process_df(csv_path, wikt_lang, table_lang, dscr_type, current_save_path=''):
    df = pd.read_csv(csv_path, index_col=0)
    df = process_table(df)
    df = add_table_wikt_cols(df, wikt_lang, table_lang, dscr_type)
    df = add_table_id(df, wikt_lang, table_lang, dscr_type)
    df = sort_columns(df, start_cols=['word_id', 'word', 'pos', 'lang_code'], end_cols=['dsrc'], col_end_sort='word_id')
    if current_save_path != '':
        fa = wikt_lang[0].upper()
        fa = fa + table_lang[0].upper()
        fa = fa + dscr_type
        fl = dscr_type.lower()
        file_name = f"stripped_sorted_{fa}"
        folder_name = f"{fl}_table_lang" 
        df.to_csv(Path(current_save_path, wikt_lang, folder_name, file_name))
    return df

## NL PP Wikt

In [97]:
pp_nlwikt_nl_df = pd.read_csv(pp_nlwikt_nl_path, index_col=0)

C:\Users\elise\AppData\Local\Temp\ipykernel_13200\2371351619.py:1: DtypeWarning: Columns (22,23,24,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  pp_nlwikt_nl_df = pd.read_csv(pp_nlwikt_nl_path, index_col=0)


In [ ]:
pp_nlwikt_nl_df

In [121]:
nlwikt_nl_stripped = process_table(pp_nlwikt_nl_df)

Shape before:  (305722, 28)
Shape after:  (304974, 23)


In [122]:
nlwikt_nl = add_table_wikt_cols(nlwikt_nl_stripped, 'nl', 'nl', 'P')
nlwikt_nl_ids = add_table_id(nlwikt_nl, 'nl', 'nl', 'P')

In [123]:
nlwikt_nl_ids_sort = sort_columns(nlwikt_nl_ids, start_cols=['word_id', 'word', 'pos', 'lang_code'], end_cols=['dsrc'], col_end_sort='word_id')

['NNP_id', 'word', 'pos', 'lang_code', 'antonyms', 'categories', 'derived', 'descendants', 'etymology_texts', 'forms', 'holonyms', 'homophones', 'hypernyms', 'hyphenations', 'hyponyms', 'metonyms', 'notes', 'paronyms', 'related', 'senses', 'sounds', 'synonyms', 'tags', 'translations', 'dsrc']


In [125]:
nlwikt_nl_ids_sort.to_csv(Path(nl_nl_pp_folder, 'stripped_sorted_NNP.csv'))

In [124]:
nlwikt_nl_ids_sort

,NNP_id,word,pos,lang_code,antonyms,categories,derived,descendants,etymology_texts,forms,...,metonyms,notes,paronyms,related,senses,sounds,synonyms,tags,translations,dsrc
0,0,'k,pron,nl,NaN,"[{'name': 'Ontbrekend geluid', 'kind': 'other'...",NaN,NaN,"['Oud indo-europese stam, op te vatten als (ve...",NaN,...,NaN,NaN,NaN,NaN,[{'glosses': ['klitische eerste persoon enkelv...,"[{'audio': ""nl-'k.ogg"", 'ogg_url': ""https://co...",[{'word': 'ik'}],['person'],NaN,NNP_src
1,1,'m,pron,nl,NaN,"[{'name': 'Ontbrekend geluid', 'kind': 'other'...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[{'glosses': ['clitische voorwerpsvorm voor de...,"[{'audio': ""nl-'m.ogg"", 'ogg_url': ""https://co...",NaN,['person'],NaN,NNP_src
2,2,'n,article,nl,NaN,"[{'name': 'Lidwoord in het Nederlands', 'kind'...","[{'word': ""zo'n""}]",NaN,"['van een, waarbij de klinker ee door een apos...",NaN,...,NaN,NaN,NaN,NaN,[{'glosses': ['clitische vorm van het onbepaal...,"[{'audio': ""nl-'n.ogg"", 'ogg_url': ""https://co...",NaN,NaN,NaN,NNP_src
3,3,'nieuwe' tweezaadlobbigen,noun,nl,NaN,"[{'name': 'Frase in het Nederlands', 'kind': '...",NaN,NaN,['verbinding van\u2009 nieuwe en tweezaadlobbi...,NaN,...,NaN,NaN,NaN,NaN,[{'glosses': ['De 23e druk van de Heukels gebr...,"[{'audio': ""nl-'nieuwe' tweezaadlobbigen.ogg"",...",NaN,['plural'],NaN,NNP_src
4,4,'r,pron,nl,NaN,"[{'name': 'Ontbrekend geluid', 'kind': 'other'...",NaN,NaN,"['[voornaamwoord] (verkorting) van haar', '[bi...",NaN,...,NaN,NaN,NaN,NaN,[{'glosses': ['onbenadrukte vorm accusatief va...,"[{'audio': ""nl-'r.ogg"", 'ogg_url': ""https://co...",NaN,['person'],NaN,NNP_src
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304969,304137,überseksueel,adj,nl,NaN,[{'name': 'Bijvoeglijk naamwoord in het Nederl...,NaN,NaN,['bn: intensiverende afleiding seksueel bn met...,"[{'form': 'überseksueler', 'tags': ['uninflect...",...,NaN,NaN,NaN,NaN,[{'glosses': ['aantrekkelijk als man door een ...,"[{'audio': 'nl-überseksueel.ogg', 'ogg_url': '...",NaN,NaN,NaN,NNP_src
304970,304138,überseksueelst,adj,nl,NaN,[{'name': 'Achtervoegsel -st in het Nederlands...,NaN,NaN,['überseksueel met het achtervoegsel -st'],NaN,...,NaN,NaN,NaN,NaN,[{'glosses': ['onverbogen vorm van de overtref...,"[{'audio': 'nl-überseksueelst.ogg', 'ogg_url':...",NaN,NaN,NaN,NNP_src
304971,304139,überseksuele,adj,nl,NaN,"[{'name': 'Ontbrekend geluid', 'kind': 'other'...",NaN,NaN,['überseksueel met de uitgang -e'],NaN,...,NaN,NaN,NaN,NaN,[{'glosses': ['verbogen vorm van de stellende ...,"[{'audio': 'nl-überseksuele.ogg', 'ogg_url': '...",NaN,NaN,NaN,NNP_src
304972,304140,überseksueler,adj,nl,NaN,[{'name': 'Achtervoegsel -er in het Nederlands...,NaN,NaN,['überseksueel met het achtervoegsel -er'],NaN,...,NaN,NaN,NaN,NaN,[{'glosses': ['onverbogen vorm van de vergrote...,"[{'audio': 'nl-überseksueler.ogg', 'ogg_url': ...",NaN,NaN,NaN,NNP_src


### NL-EN

In [127]:
pp_nlwikt_en_df = pd.read_csv(pp_nlwikt_en_path, index_col=0)

In [128]:
nlwikt_en_stripped = process_table(pp_nlwikt_en_df)
nlwikt_en = add_table_wikt_cols(nlwikt_en_stripped, 'nl', 'en', 'P')
nlwikt_en_ids = add_table_id(nlwikt_en, 'nl', 'en', 'P')

Shape before:  (8720, 27)
Shape after:  (8538, 22)


In [129]:
nlwikt_en_ids_sort = sort_columns(nlwikt_en_ids, start_cols=['word_id', 'word', 'pos', 'lang_code'], end_cols=['dsrc'], col_end_sort='word_id')

['NEP_id', 'word', 'pos', 'lang_code', 'antonyms', 'categories', 'derived', 'descendants', 'etymology_texts', 'forms', 'holonyms', 'homophones', 'hypernyms', 'hyphenations', 'hyponyms', 'metonyms', 'notes', 'related', 'senses', 'sounds', 'synonyms', 'tags', 'translations', 'dsrc']


In [130]:
nlwikt_en_ids_sort.to_csv(Path(nl_folder, 'pp_en', 'stripped_sorted_NEP.csv'))

## EN PP Wikt

### EN-NL

In [131]:
pp_enwikt_nl_df = pd.read_csv(pp_enwikt_nl_path, index_col=0)

C:\Users\elise\AppData\Local\Temp\ipykernel_13200\4141469257.py:1: DtypeWarning: Columns (23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  pp_enwikt_nl_df = pd.read_csv(pp_enwikt_nl_path, index_col=0)


In [23]:
pp_enwikt_nl_df.columns

Index(['pos', 'head_templates', 'inflection_templates', 'forms', 'descendants',
       'sounds', 'hyphenations', 'etymology_text', 'etymology_templates',
       'word', 'lang', 'lang_code', 'senses', 'etymology_number', 'derived',
       'related', 'antonyms', 'wikipedia', 'hypernyms', 'synonyms',
       'categories', 'coordinate_terms', 'hyponyms', 'abbreviations',
       'holonyms', 'meronyms'],
      dtype='object')

In [132]:
pp_en_en_proc = process_table(pp_enwikt_nl_df)

Shape before:  (70461, 26)
Shape after:  (70134, 22)


In [133]:
enwikt_nl = add_table_wikt_cols(pp_en_en_proc, 'en', 'nl', 'P')
enwikt_nl_ids = add_table_id(enwikt_nl, 'en', 'nl', 'P')
enwikt_nl_ids_sort = sort_columns(enwikt_nl_ids, start_cols=['word_id', 'word', 'pos', 'lang_code'], end_cols=['dsrc'], col_end_sort='word_id')

['ENP_id', 'word', 'pos', 'lang_code', 'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants', 'etymology_templates', 'etymology_text', 'forms', 'head_templates', 'holonyms', 'hypernyms', 'hyphenations', 'hyponyms', 'inflection_templates', 'meronyms', 'related', 'senses', 'sounds', 'synonyms', 'dsrc']


In [134]:
enwikt_nl_ids_sort.to_csv(Path(current_save_folder, 'en', 'pp_nl', 'stripped_sort_ENP.csv'))

### EN-EN

In [145]:
df = pd.read_csv(pp_enwikt_en_path, index_col=0)


C:\Users\elise\AppData\Local\Temp\ipykernel_13200\2064984806.py:1: DtypeWarning: Columns (18,19,20,24,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(pp_enwikt_en_path, index_col=0)


In [147]:
df = process_table(df)
df = add_table_wikt_cols(df, 'en', 'en', 'P')

Shape before:  (703983, 28)
Shape after:  (703983, 28)


In [ ]:
df = add_table_id(df, 'en', 'en', 'P')
#

In [149]:
df = sort_columns(df, start_cols=['word_id', 'word', 'pos', 'lang_code'], end_cols=['dsrc'], col_end_sort='word_id')

['EEP_id', 'word', 'pos', 'lang_code', 'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants', 'etymology_templates', 'etymology_text', 'forms', 'head_templates', 'holonyms', 'hypernyms', 'hyphenation', 'hyphenations', 'hyponyms', 'inflection_templates', 'instances', 'meronyms', 'original_title', 'related', 'senses', 'sounds', 'synonyms', 'translations', 'troponyms', 'dsrc']


In [150]:
df

,EEP_id,word,pos,lang_code,antonyms,categories,coordinate_terms,derived,descendants,etymology_templates,...,instances,meronyms,original_title,related,senses,sounds,synonyms,translations,troponyms,dsrc
0,0,'00s,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[{'examples': [{'text': 'Since first tossing i...,NaN,NaN,NaN,NaN,EEP_src
1,1,'30s,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[{'links': [['30s', '30s#English']], 'glosses'...",NaN,NaN,NaN,NaN,EEP_src
2,2,'40s,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[{'links': [['40s', '40s#English']], 'glosses'...",NaN,NaN,NaN,NaN,EEP_src
3,3,'50s progression,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[{'examples': [{'text': '\'50s progression in ...,NaN,NaN,NaN,NaN,EEP_src
4,4,'50s progressions,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[{'links': [[""'50s progression"", ""'50s progres...",NaN,NaN,NaN,NaN,EEP_src
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703978,695929,überpopular,adj,en,NaN,NaN,NaN,NaN,NaN,"[{'name': 'af', 'args': {'1': 'en', '2': 'über...",...,NaN,NaN,NaN,NaN,[{'examples': [{'text': '“The peril is you hav...,NaN,NaN,NaN,NaN,EEP_src
703979,695930,übersexual,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[{'examples': [{'text': '2005 October 10, Andr...",NaN,NaN,NaN,NaN,EEP_src
703980,695931,ümlauts,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[{'links': [['ümlaut', 'ümlaut#English']], 'gl...",NaN,NaN,NaN,NaN,EEP_src
703981,695932,üntermensch,noun,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[{'links': [['Üntermensch', 'Üntermensch#Engli...",NaN,NaN,NaN,NaN,EEP_src


In [143]:
df

,EEP_id,word,pos,head_templates,forms,sounds,hyphenations,etymology_text,etymology_templates,lang_code,...,antonyms,descendants,holonyms,coordinate_terms,instances,troponyms,hyphenation,inflection_templates,original_title,dsrc
0,549954,'00s,noun,"[{'name': 'en-noun', 'args': {'1': 'p'}, 'expa...",NaN,NaN,NaN,NaN,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src
1,565554,'30s,noun,"[{'name': 'en-noun', 'args': {'1': 'p'}, 'expa...",NaN,NaN,NaN,NaN,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src
2,565553,'40s,noun,"[{'name': 'en-noun', 'args': {'1': 'p'}, 'expa...",NaN,NaN,NaN,NaN,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src
3,345445,'50s progression,noun,"[{'name': 'en-noun', 'args': {}, 'expansion': ...","[{'form': ""'50s progressions"", 'tags': ['plura...",NaN,NaN,NaN,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src
4,201696,'50s progressions,noun,"[{'name': 'head', 'args': {'1': 'en', '2': 'no...",NaN,NaN,NaN,NaN,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703983,463131,NaN,name,"[{'name': 'en-prop', 'args': {}, 'expansion': ...",NaN,NaN,NaN,NaN,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src
703984,463131,NaN,adj,"[{'name': 'en-adj', 'args': {}, 'expansion': '...","[{'form': 'more null', 'tags': ['comparative']...","[{'tags': ['Canada', 'Received-Pronunciation']...",NaN,"Borrowed from Middle French nul, from Latin nū...","[{'name': 'bor', 'args': {'1': 'en', '2': 'frm...",en,...,"[{'word': 'antinull'}, {'word': 'non-null'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src
703985,463131,NaN,noun,"[{'name': 'en-noun', 'args': {'1': '-'}, 'expa...","[{'form': 'N/A', 'tags': ['alternative']}, {'f...",NaN,NaN,"From Latin non + applicabilis, from applicō (“...","[{'name': 'uder', 'args': {'1': 'en', '2': 'la...",en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src
703986,463131,NaN,noun,"[{'name': 'en-noun', 'args': {}, 'expansion': ...","[{'form': 'nans', 'tags': ['plural']}]","[{'ipa': '/næn/'}, {'audio': 'en-us-nan.ogg', ...",NaN,See at naan.,NaN,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEP_src


In [151]:
df.to_csv(Path(current_save_folder, 'en', 'pp_en', 'stripped_sort_EEP.csv'))

### Raw

In [153]:
df = full_process_df(r_enwikt_en_df_path, 'en', 'en', 'R')

C:\Users\elise\AppData\Local\Temp\ipykernel_13200\1921409906.py:2: DtypeWarning: Columns (21,24,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, index_col=0)


Shape before:  (712069, 35)
Shape after:  (704217, 27)
['EER_id', 'word', 'pos', 'lang_code', 'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants', 'etymology_templates', 'etymology_text', 'forms', 'head_templates', 'holonyms', 'hypernyms', 'hyphenation', 'hyphenations', 'hyponyms', 'inflection_templates', 'instances', 'meronyms', 'original_title', 'related', 'senses', 'sounds', 'synonyms', 'translations', 'troponyms', 'dsrc']


In [154]:
df.to_csv(Path(current_save_folder, 'en', 'r_en', 'stripped_sort_EER.csv'))

In [155]:
df = full_process_df(r_enwikt_nl_df_path, 'en', 'nl', 'R')

C:\Users\elise\AppData\Local\Temp\ipykernel_13200\1921409906.py:2: DtypeWarning: Columns (18,21,24,28,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, index_col=0)


Shape before:  (70640, 35)
Shape after:  (70331, 27)
['ENR_id', 'word', 'pos', 'lang_code', 'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants', 'etymology_templates', 'etymology_text', 'forms', 'head_templates', 'holonyms', 'hypernyms', 'hyphenation', 'hyphenations', 'hyponyms', 'inflection_templates', 'instances', 'meronyms', 'original_title', 'related', 'senses', 'sounds', 'synonyms', 'translations', 'troponyms', 'dsrc']


In [156]:
df.to_csv(Path(current_save_folder, 'en', 'r_nl', 'stripped_sort_ENR.csv'))

In [157]:
nl_nl_df = full_process_df(r_nlwikt_nl_df_path, 'nl', 'nl', 'R')

C:\Users\elise\AppData\Local\Temp\ipykernel_13200\1921409906.py:2: DtypeWarning: Columns (19,23,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, index_col=0)


Shape before:  (305659, 28)
Shape after:  (304893, 23)
['NNR_id', 'word', 'pos', 'lang_code', 'antonyms', 'categories', 'derived', 'descendants', 'etymology_texts', 'forms', 'holonyms', 'homophones', 'hypernyms', 'hyphenations', 'hyponyms', 'metonyms', 'notes', 'paronyms', 'related', 'senses', 'sounds', 'synonyms', 'tags', 'translations', 'dsrc']


In [158]:
nl_nl_df.to_csv(Path(current_save_folder, 'nl', 'r_nl', 'stripped_sort_NNR.csv'))

In [159]:
nl_en_df = full_process_df(r_nlwikt_en_df_path, 'nl', 'en', 'R')


Shape before:  (8760, 28)
Shape after:  (8583, 23)
['NER_id', 'word', 'pos', 'lang_code', 'antonyms', 'categories', 'derived', 'descendants', 'etymology_texts', 'forms', 'holonyms', 'homophones', 'hypernyms', 'hyphenations', 'hyponyms', 'metonyms', 'notes', 'paronyms', 'related', 'senses', 'sounds', 'synonyms', 'tags', 'translations', 'dsrc']


In [160]:
nl_en_df.to_csv(Path(current_save_folder, 'nl', 'r_en', 'stripped_sort_NER.csv'))

## Other

In [ ]:
def transform_df_to_col_table(origin_df, col_name, group_key='word_id'):
    dropped_df = origin_df[~origin_df[col_name].isna()]

    
    words_df = dropped_df.loc[:, ['word_id', 'word']]
    df = dropped_df.loc[:, ['word_id', 'pos', col_name]]
    df[col_name] = df[col_name].apply(safe_eval)
    df = df.explode(col_name, ignore_index=True)
    
    df = pd.concat([df.drop(columns=col_name), pd.json_normalize(df[col_name])],axis=1)
    
    df = df.groupby(group_key, as_index=False).aggregate('first').reindex(columns=df.columns)
    if 'word' in df.columns:
        df = df.rename(columns={'word': col_name})
    
    df = df.merge(words_df, on='word_id')
    df_cols = df.columns.tolist()
    df_cols = df_cols[0:2] + ['word'] + df_cols[2:-1]
    df = df[df_cols]
    
    return df